In [0]:
# Databricks PySpark: read Cosmos DB (using the spark connector) and write to Azure SQL
cosmos_config = {
  "spark.cosmos.accountEndpoint": "AccountEndpoint=https://aruncosmosdb.documents.azure.com:443/;AccountKey=EV1rrfKJLqzgP2CAXd0y7aW9gPfLsZ5mzeGRWJvKl3eODKIrd9rlmQUZZN5hGdJsJwaVldcd3mQOACDbX2xB8A==;",
  "spark.cosmos.database": "BankDB",
  "spark.cosmos.container": "ATMtransactions"
}

df = spark.read.format("cosmos.oltp").options(**cosmos_config).load()

# normalize columns
from pyspark.sql.functions import col, to_date, expr
df2 = df.select(
    col("customerId").alias("CustomerID"),
    col("eventType").alias("ActivityType"),
    to_date(col("eventTime")).alias("EventDate"),
    col("eventTime").alias("EventUTC"),
    col("deviceId").alias("DeviceID"),
    col("ip").alias("IPAddress")
)

# write to Azure SQL (JDBC) staging table
jdbc_url = "Driver={ODBC Driver 18 for SQL Server};Server=tcp:arunsqlserver.database.windows.net,1433;Database=arun;Uid=arunsqlserver;Pwd=mummy143A@;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
properties = {"user":"arunsqlserver", "password":"mummy143A@", "encrypt":"true", "trustServerCertificate":"false"}
df2.write.jdbc(url=jdbc_url, table="stg.FactCustomerActivity_STG", mode="append", properties=properties)